## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1193. Monthly Transactions I (Medium)**

**Table: Transactions**

| Column Name   | Type    |
|---------------|---------|
| id            | int     |
| country       | varchar |
| state         | enum    |
| amount        | int     |
| trans_date    | date    |

id is the primary key of this table.
The table has information about incoming transactions.
The state column is an enum of type ["approved", "declined"].
 
**Write an solution to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.**

Return the result table in any order.

The query result format is in the following example.

**Example 1:**

**Input:**
**Transactions table:**
| id   | country | state    | amount | trans_date |
|------|---------|----------|--------|------------|
| 121  | US      | approved | 1000   | 2018-12-18 |
| 122  | US      | declined | 2000   | 2018-12-19 |
| 123  | US      | approved | 2000   | 2019-01-01 |
| 124  | DE      | approved | 2000   | 2019-01-07 |

**Output:**
| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
|----------|---------|-------------|----------------|--------------------|-----------------------|
| 2018-12  | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01  | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |


In [0]:
transactions_data_1193 = [
    (121, "US", "approved", 1000, "2018-12-18"),
    (122, "US", "declined", 2000, "2018-12-19"),
    (123, "US", "approved", 2000, "2019-01-01"),
    (124, "DE", "approved", 2000, "2019-01-07")
]

transactions_columns_1193 = ["id", "country", "state", "amount", "trans_date"]
transactions_df_1193 = spark.createDataFrame(transactions_data_1193, transactions_columns_1193)
transactions_df_1193.show()

+---+-------+--------+------+----------+
| id|country|   state|amount|trans_date|
+---+-------+--------+------+----------+
|121|     US|approved|  1000|2018-12-18|
|122|     US|declined|  2000|2018-12-19|
|123|     US|approved|  2000|2019-01-01|
|124|     DE|approved|  2000|2019-01-07|
+---+-------+--------+------+----------+



In [0]:
df_with_month_1193 = transactions_df_1193.withColumn("month", date_format(to_date("trans_date", "yyyy-MM-dd"), "yyyy-MM"))

In [0]:
df_with_month_1193\
    .groupBy("month", "country").agg(
    count("*").alias("trans_count"),
    sum(when(col("state") == "approved", 1).otherwise(0)).alias("approved_count"),    
    sum("amount").alias("trans_total_amount"),
    sum(when(col("state") == "approved", col("amount")).otherwise(0)).alias("approved_total_amount")
).show()

+-------+-------+-----------+--------------+------------------+---------------------+
|  month|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+-------+-------+-----------+--------------+------------------+---------------------+
|2018-12|     US|          2|             1|              3000|                 1000|
|2019-01|     US|          1|             1|              2000|                 2000|
|2019-01|     DE|          1|             1|              2000|                 2000|
+-------+-------+-----------+--------------+------------------+---------------------+

